1.pip install keras_bert
2.图片路径：input/input0

In [1]:
import pandas as pd
import numpy as np
import codecs
import os

from keras.models import load_model
from keras.preprocessing import image
#有用户自定义层，需要使用get_custom_objects来加载
from keras_bert import get_custom_objects,Tokenizer

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
model = load_model("40epochs_7_version_batchsize=24.h5",custom_objects=get_custom_objects())








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
BATCH_SIZE = 16
maxlen = 300 #最开始是200

config_path = '/openbayes/input/input1/BERT/cased_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '/openbayes/input/input1/BERT/cased_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '/openbayes/input/input1/BERT/cased_L-12_H-768_A-12/vocab.txt'

In [14]:
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])

token_dict = {}

with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # space类用未经训练的[unused1]表示
            else:
                R.append('[UNK]') # 剩余的字符是[UNK]
        return R

tokenizer = OurTokenizer(token_dict)

class data_generator:
    def __init__(self, data, batch_size=BATCH_SIZE):
        self.data = data
        self.batch_size = batch_size
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = list(range(len(self.data)))
            np.random.shuffle(idxs)
            X1, X2, X3, Y = [], [], [], []
            for i in idxs:
                d = self.data[i]
                text = str(d[1])[:maxlen]
                x1, x2 = tokenizer.encode(first=text)
                imagePath = os.path.join(image_path,d[1])
                img = image.load_img(imagePath, target_size=(224, 224))
                x3 = image.img_to_array(img)
                y = d[0]
                X1.append(x1)
                X2.append(x2)
                X3.append(x3)
                Y.append([y])               
                if len(X1) == self.batch_size or i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    X3 = np.array(X3, dtype="float") / 255.0
                    Y = seq_padding(Y)
#                     Y = np.array(Y)
#                     Y = to_categorical(Y, num_classes=CLASS_NUM)
                    yield [X1, X2, X3], Y
                    [X1, X2, X3, Y] = [], [], [], []

In [15]:
image_path = r'/openbayes/input/input0'

test_text_path = r'/openbayes/home/processed_test.csv'
test_text_data = pd.read_csv(test_text_path)
test_text_data = np.array(test_text_data)
test_text_data = test_text_data.tolist()

In [16]:
test_x = data_generator(test_text_data)
#result = model.evaluate(test_x.__iter__(),steps = len(test_x))
#predicts = model.predict(test_x, verbose=True).argmax(axis=-1)

In [17]:
predicts = model.predict(test_x.__iter__(),steps = len(test_x))

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [20]:
#pd.DataFrame(predicts.argmax(axis=-1)).to_csv("predict_task_A.csv",index=False)

In [21]:
#还原为文字
def taskA(df):
    df_copy=df.copy(deep=True)
    i=0
    while i < len(df):
        if df_copy[0][i]==1:
            df[0][i]='positive'
        elif df_copy[0][i]==2:
            df[0][i]='negative'
        elif df_copy[0][i]==0:
            df[0][i]='neutral'
        else:
            print(i)
        i=i+1
    return df

In [22]:
final_result=taskA(pd.DataFrame(predicts.argmax(axis=-1)))

In [24]:
final_result.to_csv("predict_task_A.csv",index=False)